In [95]:
import schedule
import urllib.request, json
import time
import pandas as pd
from binance import Client
import creds

api_key = 'YG568eGXIhKU3LiO6Vpw0f1W4a3ubPDWYfpfK0Q9aR1ieBuHQjbQY07DoxhcxJQj'
api_secret = 'VbyUSTKhTw7UnE64zJ7P7sZ9YnbEZ2852dktQav6UNc8U6Vj7UafuEjTvYmYWhGV'
client = Client(api_key, api_secret)

In [112]:
bars = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, "1 minute ago UTC")
for line in bars:
    del line[11:]
df = pd.DataFrame(bars, columns=['open_time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume'])

df['open_price'] = df['open_price'].astype(float)
df['high_price'] = df['high_price'].astype(float)
df['low_price'] = df['low_price'].astype(float)
df['close_price'] = df['close_price'].astype(float)
df['volume'] = df['volume'].astype(float)
df['quote_asset_volume'] = df['quote_asset_volume'].astype(float)
df['taker_buy_quote_asset_volume'] = df['taker_buy_quote_asset_volume'].astype(float)
df['taker_buy_base_asset_volume'] = df['taker_buy_base_asset_volume'].astype(float)
df.close_time = pd.to_datetime(df.open_time, unit = 'ms')
df.open_time = pd.to_datetime(df.close_time, unit = 'ms')
df.tail()

,open_time,open_price,high_price,low_price,close_price,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,2023-09-25 09:11:00,26126.16,26126.17,26126.16,26126.16,8.69468,2023-09-25 09:11:00,227158.60611,182,0.52816,13798.797947


In [113]:
import datetime
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.engine import URL


driver= 'postgresql'
user= 'postgres'
passw= 123456
host= 'localhost'
port= 5432
database= 'opa_binance'


url = URL.create(
        drivername=driver,
        username=user,
        password=passw,
        host=host,
        port=port,
        database=database
    )

engine = create_engine(url= url)


df.to_sql('streaming_data', con= engine, index=False, if_exists='append')
engine.dispose()

print(datetime.datetime.now())
print('Amount of currencies: ' + str(len(df)))
print('BTC Price', df['close_price'][0], "was inserted.")

2023-09-25 11:11:51.166083
Amount of currencies: 1
BTC Price 26126.16 was inserted.


In [111]:
print(df['close_price'][0])

26130.08
